In [11]:
import pandas as pd
import xlrd
import mip
import networkx as nx
import datetime

# definizioni costanti fittizzie

COUNTED_PASSENGERS = 0
EXTRA_PASSENGERS = 1
INITIAL_STOP = 2
FINAL_STOP = 3
INITIAL_TIME = 4
FINAL_TIME = 5
K_AUTOBUS = 33

# lettura files

Tratte = pd.read_excel('file_di_dati.xlsx', sheet_name="porzioni di viaggio")
t_v = pd.read_excel('file_di_dati.xlsx', sheet_name="tempi di percorrenza a vuoto")

#print(Tratte)
'print(tempi_vuoto)'

'''
Tratte = Tratte.to_numpy()
tempi_vuoto = tempi_vuoto.to_numpy()

print(Tratte)
#print(tempi_vuoto)
'''

# metto tutte le colonne in un array

counted_passengers = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='A', skiprows=[0]) 
counted_passengers = counted_passengers.values.tolist()

extra_passengers = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='B', skiprows=[0])
extra_passengers = extra_passengers.values.tolist() 
n_extra_passengers = len(extra_passengers)
n_tratte = n_extra_passengers

initial_stop = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='C', skiprows=[0]) 
initial_stop = initial_stop.values.tolist()

final_stop = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='D', skiprows=[0]) 
final_stop = final_stop.values.tolist()

initial_time = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='E', skiprows=[0]) 
initial_time = initial_time.values.tolist()

final_time = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='F', skiprows=[0]) 
final_time = final_time.values.tolist()

print('-----------------------------------------------------------')

# funzione per convertie i tempi

epoch = datetime.datetime.utcfromtimestamp(0)

def unix_time_sec(dt):
    return (dt - epoch).total_seconds()



-----------------------------------------------------------


In [12]:

# qua bisogna sistemare i nomi di tempi a vuoto
#questo for rende tutti i nomi omogenei a quelli delle righa zeresima cambiando la col zeresima

tempi_vuoto = (t_v.values.tolist())
luoghi = tempi_vuoto[0][:]
luoghi.pop(0)
n_luoghi = len(luoghi)

for i in range(n_luoghi):
    if tempi_vuoto[i][0]!=tempi_vuoto[0][i]:
            print(tempi_vuoto[i][0])
            tempi_vuoto[i][0]=tempi_vuoto[0][i]

#ribalto la matrice per potere scrivere tratte_vuoto[i][j]
for i in range(n_luoghi+1):
    for j in range(n_luoghi+1):
        if i>j:
            tempi_vuoto[j][i]=tempi_vuoto[i][j]
        if i == j:
            tempi_vuoto[i][j]=0


#print(luoghi)
n_luoghi = len(luoghi)
#print(n_luoghi)
#print(tempi_vuoto)

luoghi_to_indici = {luoghi[i]:i+1 for i in range(n_luoghi)}


# usiamo un dizionario in cui ogni nome corrisponde a un indice

#print(luoghi_to_indici)




nan
piazzale medaglie d’oro
piazzale dante


In [14]:
# i nodi sono per ogni tratta 2
# in particolare uno è la partenza e uno l' arrivo
# definisco tutti i nodi, li collgo in base alla tratta 
# definisco anche il nodo pozzo e il nodo sorgente
'''
s = 0
t = n_tratte+1

nodi_partenze = []

for i in range(1,n_tratte+1):
    nodi_partenze.append(i)

nodi_arrivi = []

for i in range(1,n_tratte+1):
    nodi_arrivi.append(i)


print(nodi_arrivi)

'''

'\ns = 0\nt = n_tratte+1\n\nnodi_partenze = []\n\nfor i in range(1,n_tratte+1):\n    nodi_partenze.append(i)\n\nnodi_arrivi = []\n\nfor i in range(1,n_tratte+1):\n    nodi_arrivi.append(i)\n\n\nprint(nodi_arrivi)\n\n'

In [8]:
# nodi = tratte
N = []
for i in range(0,n_tratte+1):
    N.append(i) #il nodo 0 è n e l'n_tratte+1 è t


In [10]:
'''
A = []
# arco s
#arco t
 
# archi compatibilità
for i in range (n_tratte):
    for j in range(n_tratte):
        if i!= j and initial_time[j] <= final_time[i] + tempi_vuoto[i][j]:
            A.append((i,j))
            print(A[i][j])


COMP = []
# archi bilancio
for i in range(len(initial_stop)):
    COMP.append((i,i)
'''
A= []
# arco s
s = 0
for j in range (n_tratte):
    A.append((s,j))
    
# archi compatibilità
for i in range(n_tratte):
    indice_luogo_corrente = luoghi_to_indici[final_stop[i][0]]
    for j in range(n_tratte):
        indice_luogo_desiderato = luoghi_to_indici[initial_stop[j][0]]
    if i!= j and initial_time[j][0] <= final_time[i][0] + tempi_vuoto[i][j]:
        A.append((i+1,j+1))
#arco t
t = n_tratte+2
for i in range(n_tratte):
    A.append((i,t))

print(A)

KeyError: 'medaglie d’oro'

In [19]:
# >>>>>> variabili <<<<<<
m=[]
P_recuperati = []

for b in range(K_AUTOBUS):
    X=[]
    m.append(mip.Model())

X={(i,j): m[b].add_var(var_type = mip.BINARY) for (i,j) in A}     # Variabile Booleana (se prendo il relativo arco (i,j))


# >>>>>> vincoli <<<<<<

for i in N:
        if i==0:
            m[b].add_constr(mip.xsum(X[i,j] for j in N if (i,j) in A) -      # Outgoing flow from i
                         mip.xsum(X[j,i] for j in N if (j,i) in A) ==        # Incoming flow into i
                         1) #1= num bus
        if i==n_tratte+1:
            m[b].add_constr(mip.xsum(X[i,j] for j in N if (i,j) in A) -      # Outgoing flow from i
                         mip.xsum(X[j,i] for j in N if (j,i) in A) ==        # Incoming flow into i
                         -1)
        if i!=0 and i!=n_tratte+1:
            m[b].add_constr(mip.xsum(X[i,j] for j in N if (i,j) in A) -      # Outgoing flow from i
                         mip.xsum(X[j,i] for j in N if (j,i) in A) ==        # Incoming flow into i
                         0)



# STAMPA della SOLUZIONE:

status = m[b].optimize()  

P_recuperati.append(m[b].objective_value)

print(f"\nIl Bus {b} raccoglie {P_recuperati[b]} passeggeri: \n")
eliminare = []
for (i,j) in A:
    if(X[i,j].x):
        print(f"prendo l'arco {i} --> {j}")
        if j == p+2:
            eliminare.append((i,j))
        else:
            for z in V:
                if (z,j) in A:
                    eliminare.append((z,j))
        
for x in eliminare:        # Ad ogni bus che "spediamo" sulla rete, togliamo i relativi archi che vengono scelti
    A.remove(x)            # Così mano a mano avremo un grafo con meno archi (e quindi meno persone extra da raccogliere)
    
tot_passeggeri += m[b].objective_value
print(f"\n--> Il Totale dei passeggeri recuperati fino ad ora è: {tot_passeggeri} \n \n")   


IndexError: list index out of range